In [ ]:
from nBestAligner.nBestAlign import align
import torch

In [ ]:
nbest = [
    [1,2,3,4,6],
    [0, 1, 2, 10],
    [2,3,5,7],
    [5,10, 4, 8],
]

In [ ]:
device = 'cuda' if  torch.cuda.is_available() else 'cpu'

In [ ]:
device = torch.device(device)

In [ ]:
align(nbest, 4)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer(
                "cyclone/simcse-chinese-roberta-wwm-ext",
                device = device
            )


In [ ]:
tokens = ['广州市房地产中介协会分析'].to(device)

In [ ]:
embedding = model.encode(tokens)

In [ ]:
embedding.shape

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
token = tokenizer.convert_tokens_to_ids(list('广州市房地产中介协会分析'))

In [ ]:
model.encode(token)

In [ ]:
import torch
import logging
from torch.nn.functional import log_softmax
from transformers import (
    BertForMaskedLM,
    AutoModelForCausalLM,
    BertTokenizerFast,
)
from torch.optim import AdamW

In [ ]:
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese')

In [ ]:
seq = torch.tensor([
    [
                101,
                2408,
                2336,
                2356,
                2791,
                1765,
                772,
                704,
                792,
                1291,
                833,
                1146,
                3358,
                102
            ],
            [
                101,
                2408,
                2336,
                2356,
                2791,
                1765,
                772,
                704,
                6381,
                1291,
                833,
                102,
                0,
                0
            ]
        ])

In [ ]:
output = model(seq)

In [ ]:
output.logits.shape

In [ ]:
non_cls_index = (seq != 101)
non_sep_index = (seq != 102)
non_pad_index = (seq != 0)

In [ ]:
temp_logit = torch.logical_and(non_cls_index, non_sep_index)
token_index = torch.logical_and(temp_logit, non_pad_index)
token_index.shape

In [ ]:
real_token = seq[token_index]

In [ ]:
seq[token_index]

In [ ]:
logit = output.logits

In [ ]:
logit.shape

In [ ]:
seq.shape

In [ ]:
score = logit.gather(2, seq.unsqueeze(2)).squeeze(-1)
score.shape

In [ ]:
true_index = torch.nonzero(token_index)

In [ ]:
sum_score = 0
last_i = -1
result = []
for i, j in true_index:
    if (last_i >= 0 and last_i != i):
        print('flush')
        result.append(sum_score)
        sum_score = 0
    print(i)
    sum_score += score[i][j].item()
    last_i = i.item()
result.append(sum_score)

In [ ]:
result

In [ ]:
exp_logit = torch.exp(logit)

In [ ]:
torch.sum(logit, -1)

In [ ]:
string = "你好啊"
list(string)

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('fnlp/bart-base-chinese')

In [ ]:
string1 = '我要吃飯要吃飯吃飯飯'
string2 = '我要吃飯#要吃飯#吃飯#飯'

In [ ]:
str_list1 = [x for x in string1]
str_list2 = [x for x in string2]

In [ ]:
tokenizer.convert_ids_to_tokens([0])

In [ ]:
print(str_list1)
print(str_list2)

In [ ]:
print(tokenizer.tokenize(string1))
print(tokenizer.tokenize(string2))

In [ ]:
str_list2 == tokenizer.tokenize(string2)

In [ ]:
tokenizer.convert_tokens_to_ids(str_list2)

In [ ]:
string = "00010002"
string = string.lstrip('0')
print(string)
split_string = [string[max(i - 4, 0) : i] for i in range(len(string), 0 , -4)]
split_string[::-1]

In [ ]:
data = [i for i in range(1,11)]
label = [i for i in range(-1, -11, -1)]

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class testData(Dataset):
    def __init__(self, data, label):
        self.input = data
        self.label = label
    def __getitem__(self, idx):
        return (
            self.input[idx], self.label[idx]
        )
    def __len__(self):
        return len(self.input)

def collate(sample):
    for s in sample:
        a = [unit for unit in s[0]]
        b = [unit for unit in s[1]]
    return a, b

In [ ]:
test = testData(data, label)
loader = DataLoader(
    dataset = testData,
    batch_size = 1,
    collate_fn = collate,
    shuffle = True
)